## <center><font color=navy>Big Data Economics</font></center>
### <center>Case Study: Recommender Systems</center>
#### <center>Ali Habibnia</center>

    
<center> Assistant Professor, Department of Economics, </center>
<center> and Division of Computational Modeling & Data Analytics at Virginia Tech</center>
<center> habibnia@vt.edu </center> 

<img src="images/recom.png" alt="Drawing" style="width: 650px;"/>



As a showcase we are going to implement a recommender system. These days we are constantly being recommended from varieties of sources to what content to read (Facebook), what music to listen to (Spotify), etc…

**Recommender system strategies**

**Content-based Filtering** → creates a profile for each user or product to characterize its nature

**Collaborative Filtering** → analyzes relationships between users and inter-dependencies among products to identify new user-item associations

Collaborative filtering is generally more accurate then content filtering however, it suffers from cold start problem. (If new user exist and does not have any inter-dependencies among others, we can’t recommend anything). In general there is two method to achieve Collaborative filtering: Neighborhood Methods and Latent Factor (Matrix factorization)

### Matrix Factorization

Here we are going to investigate Matrix Factorization. When a user gives feed back to a certain movie they saw (say they can rate from one to five), this collection of feedback can be represented in a form of a matrix. Where each row represents each users, while each column represents different movies. Obviously the matrix will be sparse since not everyone is going to watch every movies, (we all have different taste when it comes to movies).

One strength of matrix factorization is the fact that it can incorporate implicit feedback, information that are not directly given but can be derived by analyzing user behavior. Using this strength we can estimate if a user is going to like a movie that he/she never saw. And if that estimated rating is high, we can recommend that movie to the user.

<img src="images/rating_matrix.png" alt="Drawing" style="width: 600px;"/>

The above image does an excellent job of summarizing, the core idea behind matrix factorization. Let there be matrix $A$ with dimensionality of $(m,n)$ this matrix can be viewed as a dot product between two matrix with each matrices having dimensions of $(m,k)$ and $(k,n)$.

> Just as a side note, this Matrix Factorization is heavily related to Singular Value Decomposition (SVD). One downside of SVD is the fact that when the original matrix is sparse (incomplete) left and right singular vectors are undefined. So there are ways to impute missing values. Instead of using SVD we are going to use an iterative method called FunkSVD which became popular during the famous Netflix challenge. Read this interesting [New York Times article](https://www.nytimes.com/2008/11/23/magazine/23Netflix-t.html?pagewanted=all&_r=0) for more information.

$k$ is the number of latent factors. The algorithm finds $k$ **latent** characteristic for each movie and user. That is they are coding some meaningful aspect of the movies and user profiles.

Movielens is a dataset of (user, movie, rating) triples. Given training data of such triples, we want to predict ratings for unseen (user, movie) pairs. All ratings are between 0.5 and 5.0. 

First, let's download and extract the dataset:

In [4]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip
! unzip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  1647k      0 --:--:-- --:--:-- --:--:-- 1647k0     0   253k      0  0:00:03 --:--:--  0:00:03  252k
Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [5]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('ml-latest-small/movies.csv')
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')

In [6]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Let's first install torch library. This may take a minute or two:

In [ ]:
!git clone https://github.com/torch/distro.git ~/torch --recursive
!cd ~/torch; bash install-deps;
!./install.sh

Now let's define the model in PyTorch. You can see that the affinity of the user for an item (movie) is calculated by multiplying the user latent vector by the movie latent vector. 

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
      super().__init__()
      # create user embeddings
      self.user_factors = torch.nn.Embedding(n_users, n_factors)
      # create item embeddings
      self.item_factors = torch.nn.Embedding(n_items, n_factors)
      self.user_factors.weight.data.uniform_(0, 0.05)
      self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
      # matrix multiplication
      users, items = data[:,0], data[:,1]
      return (self.user_factors(users)*self.item_factors(items)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)



Then we implement a loader for our dataset. You will later see why feeding (mini-) batches of data can improve the computational performance of the training algorithm.

In [0]:
# Creating the dataloader
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

class Loader(Dataset):
  def __init__(self):
    self.ratings = ratings_df.copy()
    # Extract all user IDs and movie IDs
    users = ratings_df.userId.unique()
    movies = ratings_df.movieId.unique()

    #Producing new continuous IDs for users and movies
    self.userid2idx = {o:i for i,o in enumerate(users)}
    self.movieid2idx = {o:i for i,o in enumerate(movies)}
    
    self.idx2userid = {i:o for o,i in self.userid2idx.items()}
    self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

    self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
    self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])

    self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
    self.y = self.ratings['rating'].values
    self.x, self.y  = torch.tensor(self.x), torch.tensor(self.y)
  
  def __getitem__(self, index):
    return (self.x[index], self.y[index])
  
  def __len__(self):
    return len(self.ratings)

  
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()


n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())

print("Number of users:", n_users)
print("Number of movies:", n_items)
print("Number of ratings:", len(ratings_df))

Number of users: 610
Number of movies: 9724
Number of ratings: 100836


We have 610 users and 9724 movies. That is the full rating matrix has 5931640 elements. But we have only 100836 ratings. So only 1.69% of the matrix is filled and it is very sparse. If the matrix is stored with `float32` data type, it would only need ~22MB to be stored in RAM. But movielens also publishes much larger rating datasets in orders of millions. As the number of users and movies grow, the size of the rating matrix grows exponentially, to a extent that storing the full matrix in memory would be a challenge.

An advantage of the current method to SVD is that the rating matrix is realized implicitly and we don't need to form the full dense matrix.

#### Training

Next we want to train (fit) the model. So we had two matrices with user and movie latent factors (embeddings). That is one vector for each movie and one vector for each user. 

The matrices holding latent factors are initially filled with random variables. That is, they would not make sensible prediction if we multiply user and movie vectors to get ratings. 

Next we are going to use SGD (Stochastic Gradient Descent) to find the proper values for the latent factors. 

SGD works by changing value of variables such that value of some function is minimized. We call this function the **loss function**.  Here the loss function is how much the prediction of the rating for a known rating deviates from the actual value. By minimizing this **error**, we hope to find parameter values that will **generalize** to unseen user-movie pairs.

In [0]:
num_epochs = 15
cuda = torch.cuda.is_available()
print("Is running on GPU:", cuda)
model = MatrixFactorization(n_users, n_items, n_factors=8)
if cuda:
  model = model.cuda()
loss_fn = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(),
                            lr=1e-3)

train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

for it in tqdm(range(num_epochs)):
  losses = []
  for x, y in train_loader:
    if cuda:
      x, y = x.cuda(), y.cuda()
    optimizer.zero_grad()
    outputs = model(x)

    loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))
  
#   break

Is running on GPU: False


/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


iter #0 Loss: 11.070071899951412
iter #1 Loss: 4.7471233691055765
iter #2 Loss: 2.477282887939269
iter #3 Loss: 1.7224268771065068
iter #4 Loss: 1.346314886197221
iter #5 Loss: 1.128931223287195
iter #6 Loss: 0.9916708728837483
iter #7 Loss: 0.900546970403739
iter #8 Loss: 0.8374471417462765
iter #9 Loss: 0.7924228728876501
iter #10 Loss: 0.7594797659646436
iter #11 Loss: 0.7348613678879544
iter #12 Loss: 0.7162211461720733
iter #13 Loss: 0.7017044715469863
iter #14 Loss: 0.6905001870871801



By training the model, we will have the tuned latent factors for movies and users. 

Now what if we want to propose some movies to a user to watch. We simple take the user's latent factors and multiply them by every movie latent factors. As a result, we will have the predicted rating of the user for each movie. Then we sort those movies, in descending order or predicted rating, and report back the top items to the user as the movies that he would probably enjoy watching. Voila!

#### Clustering Movies

The ML task we investigated was an example of supervised learning. We were told the ratings of some users for some movies and we are expected to be able to predict how a user would predict an unseen movie.

Now, we are going to see an example of unsupervised learning. As we mentioned, the latent factors represent some aspects of the movies and users. So for example latent factor #1 might be an indicator of the movie being a classic movie or not. What the algorithm finds is usually more complicated than what can be put into words, but if you plot the movies based on latent factors, you may see that clearly the can distinguish movies based on some criteria that the algorithm itself finds. A side product of having latent factors is that we can find similar movies by unsupervised learning. The movies that are similar tend to have latent factors that are "closer" to each other. Based on this observation we use a famous clustering algorithm called kMeans to put movies into a set of categories.

In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.numpy()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

Next we are going to show 10 blockbuster movies in each category. It can be seen that movies that are put into the same cluster tend to have similar genre. Notice that the algorithm was never aware of the movies' name or genre and it somehow extracted this information merely by looking at a bunch of numbers representing how some users responded to some movies.

Similar to this, a web service provider may do this on users' latent factor to suggest friendship among users with similar taste.

In [0]:
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Toy Story (1995)
	 Seven (a.k.a. Se7en) (1995)
	 Apollo 13 (1995)
	 Fugitive, The (1993)
	 Lord of the Rings: The Two Towers, The (2002)
	 Aladdin (1992)
	 Sixth Sense, The (1999)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
Cluster #1
	 Batman Forever (1995)
	 Waterworld (1995)
	 Ace Ventura: When Nature Calls (1995)
	 Nutty Professor, The (1996)
	 Mission: Impossible II (2000)
	 Charlie's Angels (2000)
	 Honey, I Shrunk the Kids (1989)
	 Lost World: Jurassic Park, The (1997)
	 Austin Powers in Goldmember (2002)
	 Blair Witch Project, The (1999)
Cluster #2
	 Speed 2: Cruise Control (1997)
	 Battlefield Earth (2000)
	 Problem Child (1990)
	 Flintstones in Viva Rock Vegas, The (2000)
	 Spice World (1997)
	 Stuart Saves His Family (1995)
	 Catwoman (2004)
	 Problem Child 2 (1991)
	 Texas Chainsaw Massacre, The (2003)
	 Rollerball (2002)
Cluster #3
	 Independence Day (a.k.a. ID4) (1996)
	 Batman (1989)
	 True Lies (199